In [1]:
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [2]:
files = glob.glob("*csv")

In [3]:
dfs = []
for file in files:
    df = pd.read_csv(file, encoding="cp949")
    dfs.append(df)

In [4]:
df = pd.concat(dfs,ignore_index = True)
df = df.iloc[:,2:]
df.columns = ["times", "temp"]
df = df.sort_values("times").reset_index(drop=True)
df["times"] = pd.to_datetime(df["times"])
_df = pd.DataFrame({"times":pd.date_range(df["times"].min(),df["times"].max(), freq="h")})
df = pd.merge(df, _df,on="times",how="outer")

# df["temp"].fillna(method="bfill").fillna(method="ffill")
df["temp"] = df["temp"].ffill().bfill()
df = df.sort_values("times").reset_index(drop=True)

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))

In [6]:
scaler.fit(df.loc[df["times"].dt.year!=2025,["temp"]])

MinMaxScaler(feature_range=(-1, 1))

In [7]:
df.loc[df["times"].dt.year!=2025,"temp"] = scaler.transform(
    df.loc[df["times"].dt.year!=2025,["temp"]])
df.loc[df["times"].dt.year==2025,["temp"]] = scaler.transform(
    df.loc[df["times"].dt.year==2025,["temp"]])

In [8]:
n_lags = 24
n_forecast = 3

df_lag = df.copy()
for i in range(n_lags-1, 0, -1):
    df_lag[f'lag_{i}'] = df_lag['temp'].shift(i)
df_lag['lag_0'] = df_lag['temp']
for i in range(1, n_forecast + 1):
    df_lag[f'target_{i}h'] = df_lag['temp'].shift(-i)
df_rnn = df_lag.dropna().reset_index(drop=True)
df_rnn = df_rnn.drop("temp",axis=1)

In [9]:
tr_df = df_rnn[df_rnn["times"].dt.year!=2025]
te_df = df_rnn[df_rnn["times"].dt.year==2025]

In [10]:
tr_df, val_df = train_test_split(tr_df, test_size=0.2, random_state=42, shuffle=True)

In [11]:
tr_df = tr_df.reset_index(drop = True)
val_df = val_df.reset_index(drop = True)

In [12]:
class RNNGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, x_idx, y_idx, batch_size = 32, shuffle = True):
        self.df = df
        self.x_idx = x_idx
        self.y_idx = y_idx
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.df))   
        self.shuffle = shuffle
        self.on_epoch_end()
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))

    def __getitem__(self, idx):
        batch_idx = self.indexes[idx*self.batch_size:(idx+1)*self.batch_size]
        X = self.df.iloc[batch_idx, self.x_idx].values
        y = self.df.iloc[batch_idx, self.y_idx].values
        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

In [13]:
tr_gen=RNNGenerator(tr_df, range(1,25),range(25,28), batch_size = 32)
val_gen=RNNGenerator(val_df, range(1,25),range(25,28), batch_size = 32)

In [14]:
inputs = tf.keras.layers.Input(shape = (n_lags, 1))
x = tf.keras.layers.SimpleRNN(64, activation = "tanh")(inputs)
x = tf.keras.layers.Dense(32, activation = "relu")(x)
outputs = tf.keras.layers.Dense(n_forecast)(x)
model = tf.keras.Model(inputs, outputs)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.MeanSquaredError()
metrics = [tf.keras.metrics.MeanAbsoluteError()]
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics
)

model.summary()

2025-09-02 13:16:49.649127: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 24, 1)]           0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                4224      
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                                 
Total params: 6,403
Trainable params: 6,403
Non-trainable params: 0
_________________________________________________________________


2025-09-02 13:16:49.659353: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-09-02 13:16:49.662258: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-09-02 13:16:49.665428: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-02 13:16:49.666591: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [15]:
epochs = 10
history = model.fit(
    tr_gen,
    validation_data = val_gen, 
    epochs=epochs,
    verbose=1
)

Epoch 1/30
  2/438 [..............................] - ETA: 38s - loss: 0.2864 - mean_absolute_error: 0.4376  

2025-09-02 13:16:52.903556: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


438/438 [==============================] - 32s 68ms/step - loss: 0.0085 - mean_absolute_error: 0.0598 - val_loss: 0.0034 - val_mean_absolute_error: 0.0425
Epoch 2/30
438/438 [==============================] - 30s 68ms/step - loss: 0.0026 - mean_absolute_error: 0.0373 - val_loss: 0.0024 - val_mean_absolute_error: 0.0362
Epoch 3/30
438/438 [==============================] - 28s 64ms/step - loss: 0.0023 - mean_absolute_error: 0.0347 - val_loss: 0.0021 - val_mean_absolute_error: 0.0319
Epoch 4/30
438/438 [==============================] - 29s 67ms/step - loss: 0.0022 - mean_absolute_error: 0.0337 - val_loss: 0.0023 - val_mean_absolute_error: 0.0345
Epoch 5/30
438/438 [==============================] - 29s 66ms/step - loss: 0.0021 - mean_absolute_error: 0.0334 - val_loss: 0.0021 - val_mean_absolute_error: 0.0321
Epoch 6/30
438/438 [==============================] - 29s 67ms/step - loss: 0.0021 - mean_absolute_error: 0.0326 - val_loss: 0.0020 - val_mean_absolute_error: 0.0319
Epoch 7/30
438/


KeyboardInterrupt



In [16]:
pred = model.predict(te_df.iloc[:,1:25].values)
y_true = scaler.inverse_transform(te_df.iloc[:,25:])
y_pred = scaler.inverse_transform(pred)

In [17]:
[mean_absolute_error(y_true[:,i], y_pred[:,i]) for i in range(0,n_forecast)]

[0.5343155001111578, 0.8977738858324826, 1.2622681503102333]

In [18]:
inputs = tf.keras.layers.Input(shape = (n_lags, 1))
x = tf.keras.layers.LSTM(64, return_sequences = True)(inputs)
x = tf.keras.layers.LSTM(64)(x)
x = tf.keras.layers.Dense(32, activation = "relu")(x)
outputs = tf.keras.layers.Dense(n_forecast)(x)
model = tf.keras.Model(inputs, outputs)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.MeanSquaredError()
metrics = [tf.keras.metrics.MeanAbsoluteError()]
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics
)

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 24, 1)]           0         
                                                                 
 lstm (LSTM)                 (None, 24, 64)            16896     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 3)                 99        
                                                                 
Total params: 52,099
Trainable params: 52,099
Non-trainable params: 0
_________________________________________________________________


In [20]:
epochs = 10
history = model.fit(
    tr_gen,
    validation_data = val_gen, 
    epochs=epochs,
    verbose=1
)

Epoch 1/10
438/438 [==============================] - 10s 22ms/step - loss: 0.0060 - mean_absolute_error: 0.0591 - val_loss: 0.0044 - val_mean_absolute_error: 0.0507
Epoch 2/10
438/438 [==============================] - 9s 20ms/step - loss: 0.0036 - mean_absolute_error: 0.0451 - val_loss: 0.0029 - val_mean_absolute_error: 0.0400
Epoch 3/10
438/438 [==============================] - 9s 20ms/step - loss: 0.0028 - mean_absolute_error: 0.0388 - val_loss: 0.0024 - val_mean_absolute_error: 0.0341
Epoch 4/10
438/438 [==============================] - 9s 21ms/step - loss: 0.0024 - mean_absolute_error: 0.0356 - val_loss: 0.0026 - val_mean_absolute_error: 0.0378
Epoch 5/10
438/438 [==============================] - 9s 20ms/step - loss: 0.0023 - mean_absolute_error: 0.0345 - val_loss: 0.0022 - val_mean_absolute_error: 0.0333
Epoch 6/10
438/438 [==============================] - 9s 21ms/step - loss: 0.0022 - mean_absolute_error: 0.0340 - val_loss: 0.0022 - val_mean_absolute_error: 0.0332
Epoch 7/1

In [21]:
pred = model.predict(te_df.iloc[:,1:25].values)
y_true = scaler.inverse_transform(te_df.iloc[:,25:])
y_pred = scaler.inverse_transform(pred)

In [22]:
[mean_absolute_error(y_true[:,i], y_pred[:,i]) for i in range(0,n_forecast)]

[0.5055446644137233, 0.8486950697230231, 1.2152682513854156]

In [24]:
inputs = tf.keras.layers.Input(shape = (n_lags, 1))
x = tf.keras.layers.GRU(64, return_sequences=True)(inputs)
x = tf.keras.layers.GRU(32)(x)
x = tf.keras.layers.Dense(32, activation = "relu")(x)
outputs = tf.keras.layers.Dense(n_forecast)(x)
model = tf.keras.Model(inputs, outputs)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.MeanSquaredError()
metrics = [tf.keras.metrics.MeanAbsoluteError()]
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics
)

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 24, 1)]           0         
                                                                 
 gru_1 (GRU)                 (None, 24, 64)            12864     
                                                                 
 gru_2 (GRU)                 (None, 32)                9408      
                                                                 
 dense_4 (Dense)             (None, 32)                1056      
                                                                 
 dense_5 (Dense)             (None, 3)                 99        
                                                                 
Total params: 23,427
Trainable params: 23,427
Non-trainable params: 0
_________________________________________________________________


In [26]:
epochs = 10
history = model.fit(
    tr_gen,
    validation_data = val_gen, 
    epochs=epochs,
    verbose=1
)

Epoch 1/10
438/438 [==============================] - 9s 21ms/step - loss: 0.0036 - mean_absolute_error: 0.0428 - val_loss: 0.0033 - val_mean_absolute_error: 0.0414
Epoch 2/10
438/438 [==============================] - 8s 19ms/step - loss: 0.0029 - mean_absolute_error: 0.0385 - val_loss: 0.0029 - val_mean_absolute_error: 0.0384
Epoch 3/10
438/438 [==============================] - 8s 19ms/step - loss: 0.0027 - mean_absolute_error: 0.0375 - val_loss: 0.0025 - val_mean_absolute_error: 0.0343
Epoch 4/10
438/438 [==============================] - 9s 19ms/step - loss: 0.0026 - mean_absolute_error: 0.0362 - val_loss: 0.0024 - val_mean_absolute_error: 0.0336
Epoch 5/10
438/438 [==============================] - 9s 20ms/step - loss: 0.0024 - mean_absolute_error: 0.0351 - val_loss: 0.0027 - val_mean_absolute_error: 0.0379
Epoch 6/10
438/438 [==============================] - 9s 19ms/step - loss: 0.0023 - mean_absolute_error: 0.0343 - val_loss: 0.0023 - val_mean_absolute_error: 0.0339
Epoch 7/10

In [27]:
pred = model.predict(te_df.iloc[:,1:25].values)
y_true = scaler.inverse_transform(te_df.iloc[:,25:])
y_pred = scaler.inverse_transform(pred)

In [28]:
[mean_absolute_error(y_true[:,i], y_pred[:,i]) for i in range(0,n_forecast)]

[0.6145258032732139, 0.9185608399250393, 1.2258896314141547]